In [1]:
import pandas as pd
import glob
import os

In [2]:
def borrarORG(df, iata, lista1):
    if iata in lista1:
            df.loc[df["Nombre Solicitante"] == "ORG COURIER ARG", "Ruta Virtual"] = 700
    else:
        print("error")
    return df

In [3]:
def borrarMHTML():
    while True:
        pregunta = input("queres borrar los archivos MHTML?: S/N ").lower()
        if pregunta == "s":
            encontrar = glob.glob("*MHTML")
            if not encontrar:
                print("no hay archivos MHTML")
            else:
                for archivo in encontrar:
                    os.remove(archivo)
            break
        if pregunta == "n":
            print("no se borraron archivos")
            break
        else:
            print("Ingresa 's' o 'n'")

In [4]:
def ejecutarExcelFinalizado(iata):
    os.startfile(f"archivoUnificado{iata}.xlsx")

In [5]:
def manipularDatos(df, iata):
    # filtrar columnas
    df = df [
        (df["Motivo Descripción"] != "Retirado") & 
        (df["Motivo Descripción"] != "Entregado") &
        (df["Destino"] == iata)
    ].copy()

    # agregar valores
    df.loc[df["Peso del objeto"] >= 200, "Ruta Virtual"] = 503
    df.loc[df["Volumen"] >= 1.5, "Ruta Virtual"] = 503

    #limpiar columnas
    df["Distrito Destino"] = ""
    df["Provincia"] = ""

    #duplicados = df.duplicated(subset= "Nro. identificación pieza según cliente", keep = False)
    return df

In [6]:
def main():
    while True:
        iata = input("Ingresa el codigo IATA: ").upper()
        if len(iata) == 3:
            break
        else:
            print("error intente nuevametne")

    lista1 = ["CRD","LUQ"]

    if iata == "fma".upper():
        encontrar = glob.glob("*xlsx")
        lista = []
        for archivo in encontrar:
            leer = pd.read_excel(archivo)
            lista.append(leer)

        df = pd.concat(lista, ignore_index=True)
        df = manipularDatos(df, iata)
        df = borrarORG(df, iata, lista1)
        df.to_excel(f"archivoUnificado{iata}.xlsx", index=False)

        borrarMHTML()
        ejecutarExcelFinalizado(iata)

    if iata in lista1:
        encontrar = glob.glob("*xlsx")
        lista = []
        for archivo in encontrar:
            leer = pd.read_excel(archivo)
            lista.append(leer)

        df = pd.concat(lista, ignore_index=True)
        df = manipularDatos(df, iata)
        df = borrarORG(df, iata, lista1)
        df.to_excel(f"archivoUnificado{iata}.xlsx", index=False)

        borrarMHTML()
        ejecutarExcelFinalizado(iata)

if __name__ == "__main__": 
    main()

error intente nuevametne
